## Imports

In [ ]:
import evalseg
from evalseg.metrics import MME,MultiClassMetric
import numpy as np
import pandas as pd
from ipywidgets import interact, interact_manual, IntSlider
import os,glob

## Variables

In [ ]:
root_data = '../datasets'
only_2d_slice=True
out_root='../out'
os.makedirs(out_root,exist_ok=True)

## Run Metric over all datasets 2D

In [ ]:
for dataset_name in sorted([d for d in os.listdir(root_data) if os.path.isdir(f'{root_data}/{d}')]):
    dataset = evalseg.io.NibDataset(f'{root_data}/{dataset_name}')
    for case in sorted(dataset.get_available_ids()):
        if len(glob.glob(f'{out_root}/{dataset_name}-{case}-*')) >= (3 if only_2d_slice else 1):
            continue
        dataset.load_all_of_case(case)
        gto, gto_voxelsize = dataset.get_groundtruth(case)
        cto, cto_voxelsize = dataset.get_CT(case)
        gtmax = (gto.sum(axis=2).sum(axis=1).argmax(), gto.sum(axis=2).sum(axis=0).argmax(), gto.sum(axis=1).sum(axis=0).argmax())

        axes = ['x', 'y','z'] if only_2d_slice else ['all']
        for ax in axes:
            axi = {'all': -1, 'x': 0, 'y': 1, 'z': 2}[ax]
            frame = -1 if ax == 'all' else gtmax[axi]
            global data, gt, voxelsize
            gt, voxelsize = evalseg.geometry.slice(gto, gto_voxelsize, axi, [frame])
            ct, _ = evalseg.geometry.slice(cto, None, axi, [frame])
            data = {p: evalseg.geometry.slice(dataset.get_prediction(p, case)[0], None, axi, [frame])[0] for p in dataset.get_prediction_methods()}
            data = {'GroundTruth': gt, **data}
            out_path = f'{out_root}/{dataset_name}-{case}'
            if ax != 'all':
                out_path = f'{out_path}-{ax}-{frame}'
                ctlbl = f"{dataset_name.split('_')[1]} {case} {ax}={frame}"
                evalseg.ui.multi_plot_2d(ct, gt,  data, spacing=voxelsize,col=6, ctlbl=ctlbl, z_titles=[frame], dst=f'{out_path}-allct.png', show=False)
            else:
                pass  # TODO
            print(out_path)
            mme = evalseg.metrics.MultiClassMetric(evalseg.metrics.MME, dataset.num_labels)
            mme.set_reference(gt, voxelsize)
            res = mme.evaluate_multi(data)
            
            k = list(res.keys())[0]
            res = {'ignore it a': res[k], 'ignore it b': res[k], **res}
            evalseg.ui.plot_metric_multi(res, name=dataset_name, dst=f'{out_path}-metric.png', show=False,col=6)

            evalseg.ui.img.concat(glob.glob(f'{out_path}*.png'), out=f'{out_path}-res.png')
            [os.remove(f) for f in glob.glob(f'{out_path}-[!res]*.png')]
